In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [26]:
from datasets import load_dataset, Dataset,get_dataset_config_names,load_from_disk, Dataset, DatasetDict, concatenate_datasets
import random
import numpy as np
import random
import torch
from sklearn.utils import resample
import pandas as pd
import re

In [33]:
sets = ["xquad", "xlwic", "massive", "wikilingual_dataset","paws-x","Topic-Statements","belebele","sentiments"]

In [57]:
def datamaker(setlist, sz, lang,langlong,langfull):
  train_lang = []
  ddlang = []
  listlen = 0
  for s in setlist:
    print("Processing: " + s)
    if (lang in get_dataset_config_names('mbzuai-ugrip-statement-tuning/'+s)) or (langlong in get_dataset_config_names('mbzuai-ugrip-statement-tuning/'+s)) or (langfull in get_dataset_config_names('mbzuai-ugrip-statement-tuning/'+s)):
      if s == "Topic-Statements" or s == "belebele":
        dataset = load_dataset('mbzuai-ugrip-statement-tuning/'+s, langfull, split='train', streaming=True)
      elif s == "sentiments":
        dataset = load_dataset('mbzuai-ugrip-statement-tuning/'+s, langlong, split='train', streaming=True)
      else:
        dataset = load_dataset('mbzuai-ugrip-statement-tuning/'+s, lang, split='train', streaming=True)
      train_lang.append(dataset.take(sz))
      #listlen += 1
      print("Finished: "+s)
    else:
      print(lang+" not found in " + s)
      pass

  print("Starting Merging")

  for i in train_lang:
    ds = Dataset.from_generator(lambda: (yield from i), features=i.features)
    dd = DatasetDict({"train": ds})
    ddlang.append(dd)
  trainset = ddlang[0]["train"]
  for i in range(1,len(ddlang)-1):
    trainset = concatenate_datasets([trainset,ddlang[i]["train"]])
  return DatasetDict({"train": trainset})


In [58]:
data_zh = datamaker(sets, 750, "zh","chinese","zho_Hans")

Processing: xquad
Finished: xquad
Processing: xlwic
zh not found in xlwic
Processing: massive
Finished: massive
Processing: wikilingual_dataset
Finished: wikilingual_dataset
Processing: paws-x
Finished: paws-x
Processing: Topic-Statements
Finished: Topic-Statements
Processing: belebele
Finished: belebele
Processing: sentiments
Finished: sentiments
Starting Merging


In [60]:
data_zh["train"]

Dataset({
    features: ['is_true', 'statement', 'id', 'index', 'index_id', 'category', 'text', 'falsecat', 'label'],
    num_rows: 4500
})

In [61]:
langlist = [("zh","chinese","zho_Hans"), ("en","english","eng_Latn"),("fr","french","fra_Latn"),("vi","vietnamese","vie_Latn")]

In [64]:
langdata = []
for lang in langlist:
  langshort,langfull,langlong = lang
  print("-----------------Gathering: " + langlong)
  data = datamaker(sets,750,langshort,langfull,langlong)
  langdata.append(data)

print("--------------------------------Making Final Dataset----------------------------------------")
finalset = langdata[0]["train"]
for i in range(1,len(langdata)-1):
  finalset = concatenate_datasets([finalset,langdata[i]["train"]])


-----------------Gathering: zho_Hans
Processing: xquad
Finished: xquad
Processing: xlwic
zh not found in xlwic
Processing: massive
Finished: massive
Processing: wikilingual_dataset
Finished: wikilingual_dataset
Processing: paws-x
Finished: paws-x
Processing: Topic-Statements
Finished: Topic-Statements
Processing: belebele
Finished: belebele
Processing: sentiments
Finished: sentiments
Starting Merging
-----------------Gathering: eng_Latn
Processing: xquad
Finished: xquad
Processing: xlwic
Finished: xlwic
Processing: massive
Finished: massive
Processing: wikilingual_dataset
Finished: wikilingual_dataset
Processing: paws-x
Finished: paws-x
Processing: Topic-Statements
Finished: Topic-Statements
Processing: belebele
Finished: belebele
Processing: sentiments
Finished: sentiments
Starting Merging


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [68]:
finalset[12341]

{'is_true': 0,
 'statement': '"Il a été présenté pour la première fois au Festival du cinéma écologique de Bornéo le 30 septembre 2012, car il avait été présenté pour la première fois à Bornéo." is a duplicate of "Il a été présenté le 30 septembre 2012 au Festival de films écologiques de Bornéo, puisqu\'il s\'agissait de la première représentation à Bornéo."',
 'id': 342,
 'index': None,
 'index_id': None,
 'category': None,
 'text': None,
 'falsecat': None,
 'label': None}

In [83]:
finaldataset = []
for i in range(len(finalset)):
  thing = {}
  if finalset[i]["statement"] != None:
    thing["Text"] = finalset[i]["statement"]
  elif finalset[i]["text"] != None:
    thing["Text"] = finalset[i]["text"]
  else:
    print("missing text?",i)

  if finalset[i]["is_true"] != None:
    thing["label"] = finalset[i]["is_true"]
  elif finalset[i]["label"] != None:
    thing["label"] = finalset[i]["label"]
  else:
    print("missing label?",i)

  finaldataset.append(thing)

In [84]:
len(finaldataset)

14250

In [85]:
finaldataset[9000]

{'Text': 'Passage: "Make sure your hand is as relaxed as possible while still hitting all the notes correctly - also try not to make much extraneous motion with your fingers. This way, you will tire yourself out as little as possible. Remember there\'s no need to hit the keys with a lot of force for extra volume like on the piano. On the accordion, to get extra volume, you use the bellows with more pressure or speed."\n Question: "According to the passage, what would not be considered an accurate tip for successfully playing the accordion?"\n Answer: "For additional volume, increase the force with which you hit the keys"',
 'label': 1}